In [24]:
import pandas as pd
from test_script import get_preds
# from data_and_models import check_data, get_pred, convert_df, plot_shap_explanation
import numpy as np
import matplotlib.pyplot as plt
# from aux_functions import *
import os
import re


def process_punctuation_txt(review):
  # Обрабатываем знаки препинания
    review = re.sub(rf'(\W)', rf' \1 ', review[:-14])

    review = re.sub(rf'\s+', rf' ', review)

    for punct in '.,':
        review = re.sub(rf'([\d\{punct}]) (\{punct}) ([\d\{punct}])', rf'\1\2\3', review)

    for punct in '!?':
        review = re.sub(rf'(\{punct}) (\{punct}) (\{punct})', rf'\1\2\3', review)

    if review[-1] == ' ':
        review = review[:-1]

    if review[-1] not in '.!?':
        review += ' .'
    review += '#### []'

    return review


def add_spaces(review):
    pattern=r'(\w\w[а-яё])([А-Я]\w\w)'
    review=re.sub(pattern, r'\1 \2', review)
    return review
    


def process_reviews(reviews):
    
    reviews = reviews.apply(lambda x: re.sub(r'\s+', ' ', x).strip())
    reviews = reviews.apply(lambda x: x + '#### #### ####[]')
    reviews = reviews.apply(add_spaces)
    reviews = reviews.apply(process_punctuation_txt)
        
    return reviews.values.tolist()

In [25]:
def file_to_triplets(file):

    # if file.name.rsplit('.', 1)[1].lower() == 'txt':
    with open(file) as f:
        lines = f.read().split('####\n')
 
    lines = process_reviews(pd.Series(lines))



    with open('data/uploaded_file.txt', 'w') as f:
        for line in lines:
            f.write(line+'\n')
            # st.write(line[-1])
   
    # file.seek(0)
    print('Started inference')
    preds = get_preds(lines, model_path='model_58.pt', test_path='data/uploaded_file.txt')
    print(preds['pred_text'])

    return preds

In [26]:
preds = file_to_triplets('/Users/nik/Library/Mobile Documents/com~apple~TextEdit/Documents/sample_reviews.txt')
 

triplets = preds['pred_text'].values.tolist()
all_triplets = []
for label in triplets:
    # all_triplets.extend(eval(label))
    all_triplets.extend(label)

data = []
for triplet in all_triplets:
    at, ot, sp = triplet
    at = at.lower()
    ot = ot.lower()

    data.append([at, ot, sp, (at, ot, sp), (at, ot)])

df = pd.DataFrame(data)
df.columns = ['aspect', 'opinion', 'sentiment', 'triplet', 'aspect_opinion']

fig, ax = plt.subplots(figsize=(10, 10))
plt.title('Распределение тональности триплетов', fontsize=25, fontweight='bold')
# df['sentiment'].hist()
# plt.plot(fig)
# plt.plot()

Started inference
using device:cpu
Building SPAN-ASTE model...


3it [00:01,  2.87it/s]

{'precision': 0.0, 'recall': 0, 'f1': 0}
0                                                   []
1              [(сотрудниками, не компетентными, NEG)]
2    [(Преподаватели, интересные, POS), (Преподават...
Name: pred_text, dtype: object


Text(0.5, 1.0, 'Распределение тональности триплетов')

In [27]:
preds

,text,pred,label,pred_text,label_text
0,ОПАСНОСТЬ !!! ! БУДЪТЕ ОСТОРОЖНЫ !!! Купили ре...,[],[],[],[]
1,ЕСЛИ ВЫ РЕШИЛИ ОБРАТИТЬСЯ В GB ПОДУМАЙТЕ НЕ СЕ...,"[([56], [54, 55], NEG)]",[],"[(сотрудниками, не компетентными, NEG)]",[]
2,Обучаюсь на годовом курсе интернет - маркетинг...,"[([56], [57], POS), ([56], [59], POS), ([72], ...",[],"[(Преподаватели, интересные, POS), (Преподават...",[]


In [28]:
df

,aspect,opinion,sentiment,triplet,aspect_opinion
0,сотрудниками,не компетентными,NEG,"(сотрудниками, не компетентными, NEG)","(сотрудниками, не компетентными)"
1,преподаватели,интересные,POS,"(преподаватели, интересные, POS)","(преподаватели, интересные)"
2,преподаватели,лояльные,POS,"(преподаватели, лояльные, POS)","(преподаватели, лояльные)"
3,формат,удобный,POS,"(формат, удобный, POS)","(формат, удобный)"
4,презентации,хорошие,POS,"(презентации, хорошие, POS)","(презентации, хорошие)"
